In [1]:
!pip install psycopg2-binary pandas polars Faker
!pip install SQLAlchemy


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
# 1. Importar Librerías Necesarias
# ---------------------------------
import psycopg2
import pandas as pd
from faker import Faker
import random
from datetime import datetime

In [9]:
# 2. Configuración de la Conexión a la Base de Datos PostgreSQL (Local)
# ---------------------------------------------------------------------
DB_HOST     = "localhost"  
DB_NAME     = "prueba_pia"              # El nombre de la base de datos a la que te quieres conectar
DB_USER     = "postgres"         # El nombre de usuario de la base de datos
DB_PASSWORD = "dani1492"         # La contraseña del usuario de la base de datos
DB_PORT     = "5432"             # El puerto de tu servidor PostgreSQL (el predeterminado es 5432)

In [10]:
# 3. Función para Establecer la Conexión a la Base de Datos
# Establece y devuelve una conexión a la base de datos PostgreSQL.
# --------------------------------------------------------
def get_db_connection():
    try:
        conn = psycopg2.connect(
            host    =DB_HOST,
            database=DB_NAME,
            user    =DB_USER,
            password=DB_PASSWORD,
            port    =DB_PORT
        )
        print("Conexión a la base de datos PostgreSQL exitosa!")
        return conn
    except psycopg2.Error as e:
        print(f"Error al conectar a la base de datos PostgreSQL: {e}")
        return None

In [11]:
# 5. Poblamiento de la Tabla con 128 Registros de Ejemplo
from faker import Faker
import random
import psycopg2
from datetime import datetime
from random import randint
import unicodedata

def poblar_tablas(conn, num_records=128):
    cursor = conn.cursor()
    fake = Faker('es_CO')

    # ---------- Estudiante ----------
    print(f"Inserting {num_records} estudiantes...")
    for _ in range(num_records):
        numero_identificacion = random.randint(10**9, 10**10 - 1)  # número aleatorio de 10 dígitos
        cursor.execute("""
            INSERT INTO estudiante (numero_identificacion, nombre, nombre_2, apellido, apellido_2)
            VALUES (%s, %s, %s, %s, %s);
        """, (
            numero_identificacion,
            fake.first_name(), 
            fake.first_name() if random.random() < 0.5 else None, 
            fake.last_name(), 
            fake.last_name() if random.random() < 0.5 else None
        ))


    # ---------- Profesores (37 nombres del Excel) ----------
    datos_docentes = [
        ('ANDRÉS FELIPE', 'ISAZA', 'PIEDRAHITA'),
        ('ANGEL GABRIEL', 'DIAZ', 'MAZA'),
        ('BAYARDO EMILIO', 'CADAVID', 'GOMEZ'),  
        ('CARLOS ALBERTO', 'VALENCIA', 'HERNÁNDEZ'),
        ('CARLOS MARIO', 'MORENO', 'PANIAGUA'),  
        ('CLAUDIA MARIA', 'GIRALDO', 'FLOREZ'), 
        ('CRISTIAN', 'CORTES', 'CHICA'),  
        ('DAVID', 'PULGARIN', 'VELASQUEZ'),  
        ('ELKIN MAURICIO', 'GONZÁLEZ', 'MONTOYA'), 
        ('GERMÁN', 'SIERRA', 'PORTELA'), 
        ('HECTOR SAHIR', 'TABORDA', 'VARGAS'), 
        ('JAIME ERNESTO', 'SOTO', 'URDANETA'), 
        ('JHON JAIRO', 'RUEDA', 'ECHAVARRIA'),  
        ('JUAN CAMILO', 'HERRERA', 'PINEDA'),  
        ('JUAN CARLOS', 'RIVERA', 'SANTOS'),
        ('JUAN DAVID', 'PIEDRAHÍTA', 'GIL'),
        ('JUAN ESTEBAN', 'MORENO', 'VERA'),
        ('JUAN GONZALO', 'ZULUAGA', 'BOTERO'),
        ('JUAN LUIS', 'PALACIO', 'BEDOYA'),
        ('JULIAN', 'MORA', 'OROZCO'),
        ('JULIO PASTOR', 'RESTREPO', 'ZAPATA'),
        ('KEVIN', 'OSORNO', 'CASTILLO'),  
        ('LILIANA MARIA', 'GARCIA', 'AGUIRRE'),
        ('LUIS GUILLERMO', 'ZULETA', 'GONZALEZ'),
        ('ORALIA', 'CORTES', 'GRAJALES'),
        ('RAMIRO ANTONIO', 'LOPERA', 'SANCHEZ'),
        ('RAUL ALBERTO', 'BEDOYA', 'CASTRILLON'), 
        ('ROBERT DAVID', 'URDA', 'BENITEZ'),
        ('SANDRA MARCELA', 'GUERRERO', 'CALVACHE'),
        ('SERGIO HERNANDO', 'RUIZ', 'OBANDO'),
        ('VESNA', 'SRDANOVIC', ''), 
        ('WILLIAM', 'MONTOYA', 'DIAZ')  

    ]

    def clean_string(s):
        s = ''.join(c for c in unicodedata.normalize('NFD', s)
                    if unicodedata.category(c) != 'Mn')
        return s.lower().replace(' ', '')

    print(f"Inserting {len(datos_docentes)} profesores y usuarios...")
    for full_name, apellido1, apellido2 in datos_docentes:
        nombres_split = full_name.split()
        nombre = nombres_split[0]
        nombre_2 = nombres_split[1] if len(nombres_split) > 1 else ''

        numero_identificacion = random.randint(10**9, 10**10 - 1)

        # Insertar en profesor
        cursor.execute("""
            INSERT INTO profesor (numero_identificacion, nombre, nombre_2, apellido, apellido_2)
            VALUES (%s, %s, %s, %s, %s);
        """, (
            numero_identificacion,
            nombre,
            nombre_2,
            apellido1,
            apellido2
        ))

        # Generar correo único y contraseña
        correo_base = f"{clean_string(nombre)}.{clean_string(apellido1)}"
        correo = f"{correo_base}{random.randint(1, 1000)}@pascualbravo.edu.co"
        contrasena = f"{clean_string(nombre)}123"

        # Insertar en usuario
        cursor.execute("""
            INSERT INTO usuario (correo, contrasena)
            VALUES (%s, %s);
        """, (correo, contrasena))

    # ---------- Tipos de Proyecto -----------
    tipos_proyecto = ['PIA', 'PA', 'PE', 'PI']
    tipo_proyecto_ids = {}

    print(f"Inserting {len(tipos_proyecto)} tipos_proyecto reales...")
    for tipo in tipos_proyecto:
        cursor.execute("""
            INSERT INTO tipo_proyecto (tipo_proyecto)
            VALUES (%s)
            RETURNING id_tipo_proyecto;
        """, (tipo,))
        id_tipo = cursor.fetchone()[0]
        tipo_proyecto_ids[tipo] = id_tipo

    proyectos_con_tipos = [
        ('Manilla de registros biomédicos - Smart Health', 'PIA'),  
        ('Smart Farm', 'PIA'),  
        ('Sistema de medición IoT de temperatura para un extrusor de plástico por inducción electromagnética', 'PIA'),  
        ('Prototipo de Sistema de Información Web para la gestión de proyectos PA/PIA', 'PIA'),  
        ('Estabilizador con volante de inercia basado en un filtro kalman', 'PIA'),  
        ('Implementación de robot movil basado en arduino', 'PIA'),
        ('Robot movil de 600 vatios controlado via inalambrica', 'PIA'), 
        ('Control de navegación autonoma de una plataforma robótica', 'PIA'),  
        ('Sistema Ágil Cívica', 'PIA'),  
        ('Gestión Inmobiliaria 360', 'PIA'),  
        ('Solución Integral de Nómina con Simulador Personalizado para Empleados - NOMINAPLUS', 'PIA'), 
        ('Estampado neumático por clasificación de materiales por color', 'PIA'),  
        ('SYSARCS (sISTEMA DE ACCESO RAPIDO Y CONFIABLE Y SEGURO)', 'PIA'),  
        ('Creación de una empresa e implementar su desarrollo', 'PIA'),  
        ('Banda troqueladora de arepas', 'PIA'),  
        ('Diseño y construcción de una máquina para la elaboración de filamento, mediante el aprovechamiento del material reciclado "botellas PET" de la Institución Universitaria Pascual Bravo', 'PIA'),  
        ('InkPOS', 'PA'),  
        ('Aplicativo Web JAL Comuna 3', 'PA'),  
        ('Helados Mi familia', 'PA'),  
        ('SGIRD - Software de gestión integral de restaurantes y delivery', 'PA'),  
        ('SIMULACIÓN ESTÁTICA DE UNA ESTRUCTURA MECÁNICA', 'PA'),  
        ('BANDA TRANSPORTADORA INCLINADA', 'PA'),  
        ('Diseñar e implementar un modelo de control de acceso utilizando el teclado matricial', 'PA'),  
        ('Control de dispositivos Vía Telegram', 'PA'),  
        ('PLATAFORMA STEWART', 'PA'),  
        ('Diseño de un control programado para un sistema móvil con dos llantas, a partir de lógica combinacional (Arduino)', 'PA'),  
        ('AGENTES INTELIGENTES - IA - SOFTWARE', 'PA'),  
        ('Proyectos iniciación con Arduino', 'PA'),  
        ('Solucionando problemas de mi entorno con C#', 'PA'),  
        ('Simulación de antena HORN', 'PA'),  
        ('Diseño e implementación de un controlador PID para una cabina de secado de aire utilizada en la remoción de humedad residual en plantas deshidratadas', 'PA'),  
        ('Controlador PID para un "Levitador Neumático"', 'PA')  

    ]

    print(f"Inserting {len(proyectos_con_tipos)} proyectos reales...")
    for titulo, tipo_codigo in proyectos_con_tipos:
        tipo_id = tipo_proyecto_ids[tipo_codigo]
        titulo_corto = titulo[:50]

        # Generar fecha aleatoria de este año
        mes = randint(1, 12)
        dia = randint(1, 28)  # para evitar problemas con febrero
        fecha_aleatoria = datetime(2025, mes, dia).date()

        # Generar descripción aleatoria
        descripcion = titulo
        descripcion_corta = descripcion[:50]

        cursor.execute("""
            INSERT INTO proyecto (id_tipo, titulo, descripcion, fecha_inscripcion)
            VALUES (%s, %s, %s, %s);
        """, (
            tipo_id,
            titulo_corto,
            descripcion_corta,
            fecha_aleatoria
        ))


    # ---------- Tipo Entregable (seguro para VARCHAR(50)) ----------
    entregable = [
        'Poster', 'Informe', 'Prototipo'
    ]

    print(f"Inserting {len(entregable)} tipo_entregable...")
    for req in entregable:
        tipo_corto = req[:50]
        descripcion_corta = fake.word()[:50]

        cursor.execute("""
            INSERT INTO tipo_entregable (tipo_entregable, descripcion)
            VALUES (%s, %s);
        """, (
            tipo_corto,
            descripcion_corta 
        ))


    # ---------- Tipo Insumo ----------
    requerimientos = [
        'Proyector', 'Internet', 'Sonido', 'Pantalla',
        'Computador', 'Cable HDMI', 'Protoboard', 'Arduino', 'Sillas',
        'Mesas', 'Tomas eléctricas', 'Aire acondicionado', 'Iluminación',
        'Cámara', 'Resistencias', 'Wifi'
    ]

    print(f"Inserting {len(requerimientos)} tipo_insumo...")
    for req in requerimientos:
        tipo_corto = req[:50]
        valor_random = random.randint(1000, 50000)

        cursor.execute("""
            INSERT INTO tipo_insumo (tipo_insumo, valor)
            VALUES (%s, %s);
        """, (
            tipo_corto,
            valor_random
        ))


    # ---------- País ----------
    paises = ['Colombia', 'Argentina', 'México', 'Chile', 'Perú']
    print(f"Inserting {len(paises)} paises...")
    for idx, pais in enumerate(paises, start=1):
        cursor.execute("""
            INSERT INTO pais (id_pais, pais)
            VALUES (%s, %s);
        """, (idx, pais))

    # ---------- Taller Tránsito ----------
    etapas_transito = [
        'PREINSCRIPCIÓN',
        'INSCRIPCIÓN',
        'CORRECCIONES',
        'PRESENTACIÓN',
        'PITCH',
        'IMPLEMENTACIÓN'
    ]

    print(f"Inserting {len(etapas_transito)} taller_transito...")
    for etapa in etapas_transito:
        cursor.execute("""
            INSERT INTO taller_transito (nombre)
            VALUES (%s);
        """, (etapa,))

    conn.commit()
    cursor.close()
    print("¡Datos insertados exitosamente en todas las tablas!")


In [12]:
import pandas as pd

def cargar_tablas_pobladas(conn):
    tablas = [
        'estudiante', 'profesor', 'usuario', 'tipo_proyecto', 'tipo_entregable',
        'tipo_insumo', 'pais', 'taller_transito',
        'permiso', 'rol'
    ]

    dataframes = {}

    print("\nCargando tablas pobladas en Pandas...")
    for tabla in tablas:
        try:
            df = pd.read_sql_query(f"SELECT * FROM {tabla};", conn)
            dataframes[tabla] = df
            print(f"Tabla '{tabla}' cargada. Filas: {len(df)}")
        except Exception as e:
            print(f"Error al cargar tabla '{tabla}': {e}")

    print("\nTablas cargadas exitosamente. Usa 'dataframes[\"nombre_tabla\"]' para acceder a cada DataFrame.")
    return dataframes


In [13]:
def main():
    conn = None  # Inicializar la conexión
    try:
        conn = get_db_connection()
        if conn:
            # Crear todas las tablas (debes tener una función que ya incluya todos los CREATE)
            #crear_todas_las_tablas(conn)
            
            # Poblar solo las tablas recomendadas (las que llenaste con Python)
            poblar_tablas(conn, num_records=128)  # Ajusta el número si quieres menos o más registros

            # Cargar y mostrar datos con Pandas
            dataframes = cargar_tablas_pobladas(conn)
            if dataframes:
                # Ejemplo de operación: Mostrar conteo de filas por tabla
                print("\nResumen de registros cargados en Pandas:")
                for nombre_tabla, df in dataframes.items():
                    print(f"- {nombre_tabla}: {len(df)} registros")

                # Ejemplo específico: Conteo de estudiantes por nombre (u otra columna disponible)
                if 'estudiante' in dataframes:
                    print("\nConteo de estudiantes por nombre:")
                    print(dataframes['estudiante']['nombre'].value_counts().head())
    finally:
        if conn:
            conn.close()
            print("\nConexión a la base de datos cerrada.")


In [14]:
if __name__ == "__main__":
    main()


Conexión a la base de datos PostgreSQL exitosa!
Inserting 128 estudiantes...
Inserting 32 profesores y usuarios...
Inserting 4 tipos_proyecto reales...
Inserting 32 proyectos reales...
Inserting 3 tipo_entregable...
Inserting 16 tipo_insumo...
Inserting 5 paises...
Inserting 6 taller_transito...
¡Datos insertados exitosamente en todas las tablas!

Cargando tablas pobladas en Pandas...
Tabla 'estudiante' cargada. Filas: 128
Tabla 'profesor' cargada. Filas: 32
Tabla 'usuario' cargada. Filas: 32
Tabla 'tipo_proyecto' cargada. Filas: 4
Tabla 'tipo_entregable' cargada. Filas: 3
Tabla 'tipo_insumo' cargada. Filas: 16
Tabla 'pais' cargada. Filas: 5
Tabla 'taller_transito' cargada. Filas: 6
Tabla 'permiso' cargada. Filas: 0
Tabla 'rol' cargada. Filas: 0

Tablas cargadas exitosamente. Usa 'dataframes["nombre_tabla"]' para acceder a cada DataFrame.

Resumen de registros cargados en Pandas:
- estudiante: 128 registros
- profesor: 32 registros
- usuario: 32 registros
- tipo_proyecto: 4 registros
-

C:\Users\Daniela\AppData\Local\Temp\ipykernel_23716\3724583575.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(f"SELECT * FROM {tabla};", conn)
C:\Users\Daniela\AppData\Local\Temp\ipykernel_23716\3724583575.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(f"SELECT * FROM {tabla};", conn)
